In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 1

### Imports

Use `aimport` for `schedview` imports for ease of debugging.

In [ ]:
import warnings
import math
import os
import sys
import logging
from pathlib import Path
import panel as pn
import numpy as np
import pandas as pd
import param
import bokeh
from copy import deepcopy
import datetime
from pytz import timezone
import lzma
import pickle
import yaml
import json
import socket
import time
from contextlib import redirect_stdout
from pathlib import Path
from collections import OrderedDict
from tempfile import TemporaryDirectory, NamedTemporaryFile
import hashlib
import shutil

import sys
from conda.exceptions import EnvironmentLocationNotFound
from conda.gateways.disk.test import is_conda_environment
from conda.cli.main_list import print_packages

In [ ]:
from astropy.time import Time, TimeDelta
from zoneinfo import ZoneInfo
import matplotlib as mpl
import matplotlib.pyplot as plt
import hvplot.pandas

In [ ]:
import lsst.resources

In [ ]:
import rubin_scheduler
from rubin_scheduler.scheduler.example import example_scheduler
from rubin_scheduler.scheduler import sim_runner
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.sim_archive import sim_archive

In [ ]:
import schedview
import schedview.app.prenight
import schedview.compute.scheduler
import schedview.collect.opsim
from schedview.plot.visitmap import BAND_COLORS

### Further preparation of the notebook

In [ ]:
pn.extension("terminal")

### Filter warnings

Several dependencies throw prodigious instances of (benign) warnings.
Suppress them to avoid poluting the executed notebook.

In [ ]:
warnings.filterwarnings(
    "ignore",
    module="astropy.time",
    message="Numerical value without unit or explicit format passed to TimeDelta, assuming days",
)
warnings.filterwarnings(
    "ignore",
    module="pandas",
    message="In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="divide by zero encountered in divide",
)
warnings.filterwarnings(
    "ignore",
    module="healpy",
    message="invalid value encountered in multiply",
)
warnings.filterwarnings(
    "ignore",
    module="holoviews",
    message="Discarding nonzero nanoseconds in conversion.",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="invalid value encountered in arcsin",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim",
    message="All-NaN slice encountered",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_sim.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.scheduler.utils",
    message="invalid value encountered in cast",
)
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.scheduler.surveys",
    message="All-NaN slice encountered",
)

## Configuration and initial configuration

Setting `keep_rewards` to `True` results in a dashboard that includes plots of rewards.

In [ ]:
keep_rewards = True

Set the start date, scheduler, and observatory for the night:

In [ ]:
observatory = ModelObservatory()

Set `evening_mjd` to the integer calendar MJD of the local calendar day on which sunset falls on the night of interest.

In [ ]:
evening_iso8601 = "2025-01-01"

night_date = datetime.date.fromisoformat(evening_iso8601)
evening_mjd = Time(evening_iso8601).mjd
night_date, evening_mjd

If we just use this day as the start and make the simulation duration 1 day, the begin and end of the simulation will probably begin in the middle on one night and end in the middle of the next.
Instead, find the sunset and sunrise of the night we want using the almanac, and use these to determine our start time and duration.

In [ ]:
# If the date represents the local calendar date at sunset, we need to shift by the longitude in units of days
this_night = (
    np.floor(observatory.almanac.sunsets["sunset"] + observatory.site.longitude / 360)
    == evening_mjd
)

mjd_start = observatory.almanac.sunsets[this_night]["sun_n12_setting"][0].item()
mjd_end = observatory.almanac.sunsets[this_night]["sunrise"][0].item()

night_duration = mjd_end - mjd_start
time_start = Time(mjd_start, format="mjd")
time_start.iso, night_duration

In [ ]:
observatory = ModelObservatory(mjd_start=mjd_start)

In [ ]:
scheduler = example_scheduler(mjd_start=mjd_start)

## Prepare the output dir

In [ ]:
data_dir = TemporaryDirectory()
data_path = Path(data_dir.name)

## Save the starting scheduler

In [ ]:
scheduler_fname = data_path.joinpath("scheduler.pickle.xz")
scheduler.keep_rewards = True

with lzma.open(scheduler_fname, "wb", format=lzma.FORMAT_XZ) as pio:
    pickle.dump(scheduler, pio)

## Run a simulation and create the app instance

For this example, simulate starting the default first day of observing:

In [ ]:
exec_start_time = Time.now()

# Use kwargs dict rather than just assigning them in the call so the args can be included in the archive metadata
sim_runner_kwargs = {
    "mjd_start": mjd_start,
    "survey_length": night_duration,
    "record_rewards": True,
}
observatory, scheduler, observations, reward_df, obs_rewards = sim_runner(
    observatory, scheduler, **sim_runner_kwargs
)

## Save the notebook

In [ ]:
notebook_fname = data_path.joinpath("notebook.ipynb").as_posix()

In [ ]:
%notebook $notebook_fname

## Archive the results

In [ ]:
archive_files = {"scheduler": scheduler_fname, "notebook": notebook_fname}

In [ ]:
my_tags = ["test", "hen", "ducks", "geese", "oysters", "porpoises"]

In [ ]:
archive_dir = sim_archive.make_sim_archive_dir(
    observations,
    reward_df,
    obs_rewards,
    in_files=archive_files,
    tags=my_tags,
    label="Test from schedview notebook",
    sim_runner_kwargs=sim_runner_kwargs,
)
archive_dir.name

In [ ]:
!ls -alth $archive_dir.name

In [ ]:
os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"
os.environ["AWS_PROFILE"] = "prenight"

In [ ]:
archive_base_uri = "s3://rubin-scheduler-prenight/opsim/"
archive_base = lsst.resources.ResourcePath(archive_base_uri)

In [ ]:
sim_archive_resource = sim_archive.transfer_archive_dir(
    archive_dir.name, archive_base_uri=archive_base_uri
)

In [ ]:
for base_dir, found_dirs, found_files in archive_base.walk():
    for found_file in found_files:
        print(base_dir.join(found_file))

In [ ]:
sim_archive_metadata_resource = sim_archive_resource.join("sim_metadata.yaml")

read_metadata = yaml.safe_load(sim_archive_metadata_resource.read().decode("utf-8"))
read_metadata

In [ ]:
sim_archive_resource_uri = sim_archive_resource.geturl()
sim_archive_resource_uri

In [ ]:
recent_metadata = sim_archive.read_archived_sim_metadata(archive_base_uri)

In [ ]:
def pretty_metadata(metadata_dict):
    df = pd.DataFrame.from_dict(recent_metadata, orient="index")
    df.index.name = "uri"
    df.reset_index(inplace=True)
    df.set_index("label", inplace=True)
    return df


with pd.option_context("display.max_colwidth", 500):
    display(
        pretty_metadata(recent_metadata)[
            ["uri", "scheduler_version", "simulated_dates", "host", "username"]
        ]
    )

In [ ]:
obs = schedview.collect.opsim.read_opsim(sim_archive_resource)
obs